In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [75]:
df = pd.read_csv('df_training_testing.csv', encoding='utf-8')
df.head()

,video_title,video_id,moods,labeled,video_description,moods_enc
0,♩♫ Dramatic Apocalyptic Music ♪♬ - The Wastela...,5eAalHA1bAc,CHILL,t,instagram https instagram com rossbugden rossb...,1
1,Nostalgy - (No Copyright Music) Sad Cinematic ...,zViXwVZpWqY,SAD,t,download music http bit ly 2qc9e3k no copyrigh...,5
2,Under The Radar Copyright Free music for youtu...,ZTgvcaVdw5I,CHILL,t,under the radar copyright free music for youtu...,1
3,🔥Best of NCS 2017 Mix ♫ Gaming Music Mix ♫ No ...,8ZbrPz_fnXs,CONFIDENT-SASSY,t,song best of all nocopyright songs cyrse musi...,2
4,"[FREE] Lil Baby x Gunna Type Beat 2018 ""Situat...",5vSJTGpG5YM,CHILL,t,purchase link free download link https bit ly...,1


In [76]:
df.shape

(739, 6)

In [77]:
def tokenizer(text):
    return text.split()

from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

from nltk.corpus import stopwords

stop = stopwords.words('english')

In [78]:
from sklearn.model_selection import train_test_split

# X = df['video_description'].values
# X = df[['video_title', 'video_description']].values
X = df['video_title'] + ' ' + df['video_description']
y = df['moods_enc'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [79]:
import numpy as np
import re
from nltk.corpus import stopwords
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized


In [80]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',
                        n_features=2**21,
                        preprocessor=None,
                        tokenizer=tokenizer)

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

grid_search = GridSearchCV(lr, param_grid, cv=5)

X_train = vect.transform(X_train.tolist())
grid_search.fit(X_train, y_train)

/Users/sammylee/miniconda3/envs/datascience/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/sammylee/miniconda3/envs/datascience/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/sammylee/miniconda3/envs/datascience/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [83]:
X_test = vect.transform(X_test)
print('Test set accuracy: {:.3f}'.format(grid_search.score(X_test, y_test)))

Test set accuracy: 0.368


In [84]:
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

Best parameters: {'C': 10}
Best cross-validation score: 0.42
